In [1]:
#author GV KR
from keras.layers import Dense, Flatten,Lambda,Input
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
#from keras.preprocesssing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt


In [2]:
#!pip install keras.preprocessing

In [3]:
import pandas as pd

C:\Users\kmmoh\gv\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.8' currently installed).
  warnings.warn(msg, UserWarning)


In [4]:
df=pd.read_csv('train/train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
df=df.dropna()
## Get the Independent Features
X=df.drop('label',axis=1)
## Get the Dependent features
y=df['label']

In [6]:
X.shape
#y.shape

(18285, 4)

In [7]:
import tensorflow as tf
tf.__version__

'2.6.0'

In [19]:
#import vocabilary 
voc_size=5000

In [20]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense


In [21]:
messages=X.copy()
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [10]:
messages.reset_index(inplace=True)

In [11]:
import nltk
import re
from nltk.corpus import stopwords


In [12]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kmmoh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
#dataset preprocessing
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
corpus= []
for i in range(0,len(messages)) :
    #print(i)
    review = re.sub('[^a-zA-Z]',' ',messages['title'][i])
    review = review.lower()
    review = review.split()
    review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
    review=' '.join(review)
    corpus.append(review)
#corpus


In [22]:
onehot_repr=[one_hot(words,voc_size) for words in corpus]
onehot_repr

[[3383, 166, 4104, 508, 4936, 4086, 4003, 3270, 2105, 3406],
 [1856, 2574, 3699, 3921, 2280, 947, 2541],
 [1755, 4444, 3988, 386],
 [3782, 4979, 275, 4511, 766, 1830],
 [4625, 2280, 4584, 1993, 251, 1714, 2280, 2571, 1007, 4830],
 [1691,
  3771,
  4814,
  450,
  1084,
  426,
  437,
  3265,
  4529,
  1893,
  127,
  3718,
  3666,
  73,
  2541],
 [218, 832, 3703, 3200, 4744, 4849, 4028, 3846, 3875, 976, 1534],
 [4927, 1732, 2825, 3472, 3196, 3578, 426, 443, 3875, 976, 1534],
 [2970, 4110, 2208, 2749, 3647, 2411, 863, 3037, 426, 52],
 [4060, 4814, 3369, 1214, 4896, 4369, 3628, 3341],
 [4296, 1365, 1800, 1167, 959, 2439, 1515, 2663, 1196, 1705, 3830],
 [4511, 1768, 4936, 2411, 426, 3196],
 [1454, 4020, 3508, 2278, 4229, 3406, 4323, 3365, 1341],
 [2359, 2509, 3474, 2853, 3202, 4944, 4618, 3875, 976, 1534],
 [80, 373, 1656, 2928, 2522, 3875, 976, 1534],
 [3575, 120, 107, 2153, 344, 2944, 21, 4303, 2166, 1516],
 [2551, 3208, 2574],
 [4545, 930, 2119, 2688, 426, 4311, 3285, 2541],
 [4943, 676, 

#Embedding Representation

In [23]:
sentence_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sentence_length)
print(embedded_docs)

[[   0    0    0 ... 3270 2105 3406]
 [   0    0    0 ... 2280  947 2541]
 [   0    0    0 ... 4444 3988  386]
 ...
 [   0    0    0 ... 3875  976 1534]
 [   0    0    0 ... 3262 4217 1152]
 [   0    0    0 ... 3773   98 3810]]


In [24]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 3383,
        166, 4104,  508, 4936, 4086, 4003, 3270, 2105, 3406])

In [28]:
#creating a model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sentence_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())          


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [29]:
len(embedded_docs),y.shape

(18285, (18285,))

In [30]:
#import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)
X_final.shape,y_final.shape

((18285, 20), (18285,))

# Model Training

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train,y_test = train_test_split(X_final,y_final,test_size=0.33,random_state=42)
model.fit(X_train,y_train,validation_data=(X_test,y_test),epocsh=10,batch_size=64)


In [32]:
from tensorflow.keras.layers import Dropout
## creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features, input_length=sentence_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])


# Performance Metrics and Accuracy

In [36]:
y_pred=model.predict_classes(X_test)
    

In [37]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)
